In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
trainning_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor(),
)

test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor(),
)

100%|██████████| 26421880/26421880 [00:10<00:00, 2495787.30it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 188324.44it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:03<00:00, 1322672.46it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 13966543.98it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
batch_size = 64

train_dataloader = DataLoader(trainning_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X,y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()##???? transforms to a 1d tensor I believe
        self.linear_relu_stack = nn.Sequential(##????
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork().to(device)
print(model)


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()#????
    for batch, (X,y) in enumerate(dataloader):#for every batch
        X, y = X.to(device), y.to(device)#load the batch into the relevant ram

        pred = model(X)#make the prediction
        loss = loss_fn(pred, y)#calculate the loss

        loss.backward()#generate the gradients I believe
        optimizer.step()#update the weights
        optimizer.zero_grad()#???

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X) 
            print(f"loss: {loss:>7f} [{current:>5d}]/{size:>5d}")



In [7]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():#??
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()#not fully sure on this
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {100*correct:>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [8]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n----------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
----------------------------
loss: 2.302745 [   64]/60000
loss: 2.289561 [ 6464]/60000
loss: 2.274664 [12864]/60000
loss: 2.264154 [19264]/60000
loss: 2.255169 [25664]/60000
loss: 2.220179 [32064]/60000
loss: 2.228836 [38464]/60000
loss: 2.200311 [44864]/60000
loss: 2.184511 [51264]/60000
loss: 2.154082 [57664]/60000
Test Error: 
 Accuracy: 47.2%, Avg loss: 2.156655 

Epoch 2
----------------------------
loss: 2.168372 [   64]/60000
loss: 2.151173 [ 6464]/60000
loss: 2.108267 [12864]/60000
loss: 2.114793 [19264]/60000
loss: 2.068531 [25664]/60000
loss: 2.011098 [32064]/60000
loss: 2.031669 [38464]/60000
loss: 1.964567 [44864]/60000
loss: 1.953202 [51264]/60000
loss: 1.877852 [57664]/60000
Test Error: 
 Accuracy: 55.1%, Avg loss: 1.890273 

Epoch 3
----------------------------
loss: 1.927638 [   64]/60000
loss: 1.884038 [ 6464]/60000
loss: 1.795149 [12864]/60000
loss: 1.819275 [19264]/60000
loss: 1.708754 [25664]/60000
loss: 1.664061 [32064]/60000
loss: 1.677839 [38464]/60000
lo

In [9]:
torch.save(model.state_dict(), "fm.pth")
print("saved model")

saved model


In [ ]:
model = NeuralNetwork